In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# SAM 3 画像バッチ推論 (SAM 3 Image Batched Inference)

このノートブックでは、SAM 3 モデルを使用して、複数の画像に対する推論をバッチ処理（まとめて処理）する方法を解説します。
データポイントの作成、前処理、バッチ化、モデルによる推論、そして結果の視覚化までの流れをステップバイステップで学びます。

# <a target="_blank" href="https://colab.research.google.com/github/facebookresearch/sam3/blob/main/notebooks/sam3_image_batched_inference.ipynb">
#   <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
# </a>

## Google Colab セットアップ

Google Colabで実行する場合は、以下のセルを実行してください。
これにより、必要なライブラリのインストールと、データの読み込み準備が行われます。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

## 環境設定 (Environment Set-up)

まず、必要なライブラリをインストールし、環境をセットアップします。
Google Colabを使用している場合は、`using_colab = True` に設定してください。

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib scikit-learn
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam3.git'

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import sam3
from sam3.train.data.collator import collate_fn_api as collate
from sam3.model.utils.misc import copy_data_to_device
import os
sam3_root = os.path.join(os.path.dirname(sam3.__file__), "..")

In [ ]:
import torch
# Ampere GPU向けにtfloat32を有効化
# https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# ノートブック全体でbfloat16を使用。カードがサポートしていない場合はfloat16を試してください
torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

# ノートブック全体で推論モードを使用。勾配が必要な場合は無効にしてください
torch.inference_mode().__enter__()

# ユーティリティ (Utilities)

## プロット (Plotting)

このセクションには、画像の上にマスクとバウンディングボックスをプロットするためのシンプルなユーティリティが含まれています。

In [ ]:
import sys

sys.path.append(f"{sam3_root}/examples")

from sam3.visualization_utils import plot_results

## バッチ処理 (Batching)

このセクションには、データポイントを作成するためのいくつかのユーティリティ関数が含まれています。これらは必須ではありませんが、どのようにデータを作成すべきかについての良い指針となります。

以下の関数は、推論のためのデータを構築するヘルパー関数です。
- `create_empty_datapoint`: 空のデータポイントを作成します。
- `set_image`: データポイントに画像を設定します。
- `add_text_prompt`: テキストプロンプト（例：「猫」）を追加します。
- `add_visual_prompt`: ボックスプロンプト（バウンディングボックス）を追加します。

In [ ]:
from sam3.train.data.sam3_image_dataset import InferenceMetadata, FindQueryLoaded, Image as SAMImage, Datapoint
from typing import List

GLOBAL_COUNTER = 1
def create_empty_datapoint():
    """ データポイントは、複数のクエリを一度に適用できる単一の画像です。 """
    return Datapoint(find_queries=[], images=[])

def set_image(datapoint, pil_image):
    """ 処理する画像をデータポイントに追加します """
    w,h = pil_image.size
    datapoint.images = [SAMImage(data=pil_image, objects=[], size=[h,w])]

def add_text_prompt(datapoint, text_query):
    """ テキストクエリをデータポイントに追加します """

    global GLOBAL_COUNTER
    # この関数では、画像がすでに設定されている必要があります。
    # これは、マスクとボックスをリサイズする次元を把握するために画像のサイズを取得するためです。
    # 実際には、任意のサイズを設定できますが、関数の残りの部分を編集してください。
    assert len(datapoint.images) == 1, "please set the image first"

    w, h = datapoint.images[0].size
    datapoint.find_queries.append(
        FindQueryLoaded(
            query_text=text_query,
            image_id=0,
            object_ids_output=[], # 推論では未使用
            is_exhaustive=True, # 推論では未使用
            query_processing_order=0,
            inference_metadata=InferenceMetadata(
                coco_image_id=GLOBAL_COUNTER,
                original_image_id=GLOBAL_COUNTER,
                original_category_id=1,
                original_size=[w, h],
                object_id=0,
                frame_index=0,
            )
        )
    )
    GLOBAL_COUNTER += 1
    return GLOBAL_COUNTER - 1

def add_visual_prompt(datapoint, boxes:List[List[float]], labels:List[bool], text_prompt="visual"):
    """ ビジュアルクエリをデータポイントに追加します。
    bboxesはXYXY形式（左上と右下のコーナー）であることが期待されます。
    各bboxに対して、ラベル（TrueまたはFalse）が必要です。モデルは、ネガティブなものを避けながら、ポジティブなものに似たボックスを見つけようとします。
    追加のヒントとしてtext_promptを与えることもできます。必須ではありません。モデルにボックスのみに依存させたい場合は "visual" のままにしてください。

    モデルはプロンプトが一貫していることを期待することに注意してください。テキストが「象」と書かれているのに、提供されたボックスが犬を指している場合、結果は未定義になります。
    """

    global GLOBAL_COUNTER
    # この関数では、画像がすでに設定されている必要があります。
    # これは、マスクとボックスをリサイズする次元を把握するために画像のサイズを取得するためです。
    # 実際には、任意のサイズを設定できますが、関数の残りの部分を編集してください。
    assert len(datapoint.images) == 1, "please set the image first"
    assert len(boxes) > 0, "please provide at least one box"
    assert len(boxes) == len(labels), f"Expecting one label per box. Found {len(boxes)} boxes but {len(labels)} labels"
    for b in boxes:
        assert len(b) == 4, f"Boxes must have 4 coordinates, found {len(b)}"

    labels = torch.tensor(labels, dtype=torch.bool).view(-1)
    if not labels.any().item() and text_prompt=="visual":
        print("Warning: you provided no positive box, nor any text prompt. The prompt is ambiguous and the results will be undefined")
    w, h = datapoint.images[0].size
    datapoint.find_queries.append(
        FindQueryLoaded(
            query_text=text_prompt,
            image_id=0,
            object_ids_output=[], # 推論では未使用
            is_exhaustive=True, # 推論では未使用
            query_processing_order=0,
            input_bbox=torch.tensor(boxes, dtype=torch.float).view(-1,4),
            input_bbox_label=labels,
            inference_metadata=InferenceMetadata(
                coco_image_id=GLOBAL_COUNTER,
                original_image_id=GLOBAL_COUNTER,
                original_category_id=1,
                original_size=[w, h],
                object_id=0,
                frame_index=0,
            )
        )
    )
    GLOBAL_COUNTER += 1
    return GLOBAL_COUNTER - 1

# 読み込み (Loading)

まず、モデルを読み込みます。`build_sam3_image_model` 関数を使用し、BPE（Byte Pair Encoding）語彙ファイルのパスを指定します。

In [ ]:
from sam3 import build_sam3_image_model

bpe_path = f"{sam3_root}/assets/bpe_simple_vocab_16e6.txt.gz"
model = build_sam3_image_model(bpe_path=bpe_path)

次に、検証用の変換（transforms）を定義します。
ここでは、画像を1008x1008にリサイズし、テンソルに変換してから正規化を行う一連の処理を定義しています。

In [ ]:
from sam3.train.transforms.basic_for_api import ComposeAPI, RandomResizeAPI, ToTensorAPI, NormalizeAPI

from sam3.model.position_encoding import PositionEmbeddingSine
transform = ComposeAPI(
    transforms=[
        RandomResizeAPI(sizes=1008, max_size=1008, square=True, consistent_transform=False),
        ToTensorAPI(),
        NormalizeAPI(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ]
)

そして最後に、ポストプロセッサ（後処理）を定義します。
モデルの出力を処理し、マスクやバウンディングボックスを元の画像サイズに戻したり、信頼度閾値に基づいてフィルタリングしたりします。

In [ ]:
from sam3.eval.postprocessors import PostProcessImage
postprocessor = PostProcessImage(
    max_dets_per_img=-1,       # この数が正の場合、プロセッサはtopkを返します。このデモでは代わりに信頼度で制限します（下記参照）
    iou_type="segm",           # マスクが必要です
    use_original_sizes_box=True,   # ボックスは画像サイズにリサイズされるべきです
    use_original_sizes_mask=True,   # マスクは画像サイズにリサイズされるべきです
    convert_mask_to_rle=False, # ポストプロセッサはRLE形式への効率的な変換をサポートしています。このデモでは簡単なプロットのためにバイナリ形式を好みます
    detection_threshold=0.5,   # 信頼度の高い検出のみを返す
    to_cpu=False,
)

# 推論 (Inference)

推論の手順は以下の通りです：
- 上記の関数を使用して、データポイントを1つずつ作成します。作成する各クエリには一意のIDが付与され、後処理後に結果を取得するために使用されます。
- 各データポイントは、前処理変換に従って変換する必要があります（基本的には1008x1008にリサイズし、正規化します）。
- その後、すべてのデータポイントをバッチにまとめ、モデルに渡します（フォワードパス）。

### データ準備
ここでは2つの画像を準備します。
1. **画像1**: インターネット上の画像を使用し、"cat"（猫）と "laptop"（ノートPC）という2つのテキストプロンプトを与えます。
2. **画像2**: 別の画像を使用し、"pot"（鍋）というテキストプロンプトと、オーブンのダイヤルやボタンを指定するボックスプロンプトを与えます。また、ネガティブプロンプト（除外したい領域）の使用例も示します。

In [ ]:
# 画像1、2つのテキストプロンプトを使用します

img1 = Image.open(BytesIO(requests.get("http://images.cocodataset.org/val2017/000000077595.jpg").content))
datapoint1 = create_empty_datapoint()
set_image(datapoint1, img1)
id1 = add_text_prompt(datapoint1, "cat")
id2 = add_text_prompt(datapoint1, "laptop")

datapoint1 = transform(datapoint1)

In [ ]:
# 画像2、1つのテキストプロンプト、いくつかのビジュアルプロンプト
img2 = Image.open(BytesIO(requests.get("http://images.cocodataset.org/val2017/000000136466.jpg").content))

# img2 = Image.open(f"{sam3_root}/assets/images/test_image.jpg")
datapoint2 = create_empty_datapoint()
set_image(datapoint2, img2)
id3 = add_text_prompt(datapoint2, "pot")
# オーブンのダイヤルを見つけようとしています。ポジティブなボックスを与えましょう
id4 = add_visual_prompt(datapoint2, boxes=[[ 59, 144,  76, 163]], labels=[True])
# オーブンの開始/停止ボタンも取得しましょう
id5 = add_visual_prompt(datapoint2, boxes=[[ 59, 144,  76, 163],[ 87, 148, 104, 159]], labels=[True, True])
# 次に、鍋の取っ手を見つけようとします。テキストプロンプト "handle"（意図的に曖昧にしています）では、モデルはオーブンの取っ手も見つけます
# テキストクエリをより正確にすることもできますが（試してみてください！）、この例では代わりにネガティブプロンプトを活用したいと考えています
# まず、テキストプロンプトだけで何が起こるか見てみましょう
id6 = add_text_prompt(datapoint2, "handle")
# 今度は同じですが、ネガティブプロンプトを追加します
id7 = add_visual_prompt(datapoint2, boxes=[[ 40, 183, 318, 204]], labels=[False], text_prompt="handle")

datapoint2 = transform(datapoint2)

### バッチ作成
作成したデータポイントを `collate` 関数を使ってバッチにまとめ、GPU（CUDA）に転送します。

In [ ]:
# バッチにまとめてからcudaに移動
batch = collate([datapoint1, datapoint2], dict_key="dummy")["dummy"]
batch = copy_data_to_device(batch, torch.device("cuda"), non_blocking=True)

### モデル実行
バッチをモデルに入力して推論を実行し、その出力をポストプロセッサで処理して最終的な結果を取得します。

In [ ]:
# フォワードパス。最初のフォワードはコンパイルのため非常に遅くなることに注意してください
output = model(batch)

In [ ]:
processed_results = postprocessor.process_results(output, batch.find_metadatas)

# プロット (Plotting)

最後に、得られた結果（マスクとバウンディングボックス）を画像上にプロットして確認します。

In [ ]:
plot_results(processed_results, [id1, id2, id3, id4, id5, id6, id7], [img1, img1, img2, img2, img2, img2, img2])